### Deliverable 3: Optimize the Model

#### First Optimization: Remove noisy variables and add additional neurons

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN','NAME'])

In [3]:
# Look at APPLICATION_TYPE value counts for binning
app_type = application_df.APPLICATION_TYPE.value_counts()

In [4]:
# Determine which values to replace if counts are less than ...?
replace_application = list(app_type[app_type < 700].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
classification = application_df.CLASSIFICATION.value_counts()

In [6]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification[classification < 800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Generate our categorical variable lists
application_cat = list(application_df.dtypes[application_df.dtypes == 'object'].index)

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\mirua\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(columns=application_cat,axis=1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=['IS_SUCCESSFUL'],axis=1).values
y = application_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 60
hidden_nodes_layer2 = 40

nn_1 = tf.keras.models.Sequential()

# First hidden layer
nn_1.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_1.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu")
)

# Output layer
nn_1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 60)                2580      
                                                                 
 dense_1 (Dense)             (None, 40)                2440      
                                                                 
 dense_2 (Dense)             (None, 1)                 41        
                                                                 
Total params: 5,061
Trainable params: 5,061
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights_opt_1.{epoch:02d}.hdf5"

In [14]:
# Compile the model
nn_1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5
)

# Train the model
fit_model = nn_1.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5725 - accuracy: 0.7212
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5554 - accuracy: 0.7309
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5533 - accuracy: 0.7316
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5511 - accuracy: 0.7323
Epoch 5/100
790/804 [============================>.] - ETA: 0s - loss: 0.5499 - accuracy: 0.7325
Epoch 5: saving model to checkpoints\weights_opt_1.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5498 - accuracy: 0.7327
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7326
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5477 - accuracy: 0.7339
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7346
Epoch 9/100
804/804 [===========================

804/804 [==============================] - 2s 3ms/step - loss: 0.5382 - accuracy: 0.7395
Epoch 64/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5375 - accuracy: 0.7399
Epoch 65/100
803/804 [============================>.] - ETA: 0s - loss: 0.5379 - accuracy: 0.7406
Epoch 65: saving model to checkpoints\weights_opt_1.65.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5379 - accuracy: 0.7406
Epoch 66/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5373 - accuracy: 0.7402
Epoch 67/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5373 - accuracy: 0.7404
Epoch 68/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5373 - accuracy: 0.7406
Epoch 69/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5370 - accuracy: 0.7396
Epoch 70/100
803/804 [============================>.] - ETA: 0s - loss: 0.5375 - accuracy: 0.7398
Epoch 70: saving model to checkpoints\weights_opt_1.70.h

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5545 - accuracy: 0.7264 - 594ms/epoch - 2ms/step
Loss: 0.554504930973053, Accuracy: 0.7264139652252197


#### Second Optimization: Adding more hidden layers

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 60
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 20

nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu")
)

# Third hidden layer
nn_2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu")
)

# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 60)                2580      
                                                                 
 dense_4 (Dense)             (None, 40)                2440      
                                                                 
 dense_5 (Dense)             (None, 20)                820       
                                                                 
 dense_6 (Dense)             (None, 1)                 21        
                                                                 
Total params: 5,861
Trainable params: 5,861
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights_opt_2.{epoch:02d}.hdf5"

In [18]:
# Compile the model
nn_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5
)

# Train the model
fit_model = nn_2.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5749 - accuracy: 0.7187
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5574 - accuracy: 0.7304
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5538 - accuracy: 0.7301
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5518 - accuracy: 0.7318
Epoch 5/100
795/804 [============================>.] - ETA: 0s - loss: 0.5500 - accuracy: 0.7325
Epoch 5: saving model to checkpoints\weights_opt_2.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5503 - accuracy: 0.7325
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5484 - accuracy: 0.7333
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5479 - accuracy: 0.7350
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7347
Epoch 9/100
804/804 [===========================

804/804 [==============================] - 2s 2ms/step - loss: 0.5369 - accuracy: 0.7403
Epoch 64/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5371 - accuracy: 0.7404
Epoch 65/100
794/804 [============================>.] - ETA: 0s - loss: 0.5378 - accuracy: 0.7398
Epoch 65: saving model to checkpoints\weights_opt_2.65.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5370 - accuracy: 0.7402
Epoch 66/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5367 - accuracy: 0.7407
Epoch 67/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5365 - accuracy: 0.7406
Epoch 68/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5365 - accuracy: 0.7405
Epoch 69/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5363 - accuracy: 0.7407
Epoch 70/100
797/804 [============================>.] - ETA: 0s - loss: 0.5368 - accuracy: 0.7400
Epoch 70: saving model to checkpoints\weights_opt_2.70.h

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5601 - accuracy: 0.7249 - 583ms/epoch - 2ms/step
Loss: 0.5601068735122681, Accuracy: 0.7248979806900024


#### Third Optimization: Changing activation function of hidden layers or output layers and adding additional epochs

In [39]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 60
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 20

nn_3 = tf.keras.models.Sequential()

# First hidden layer
nn_3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh")
)

# Third hidden layer
nn_3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu")
)

# Output layer
nn_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_3.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_31 (Dense)            (None, 60)                2580      
                                                                 
 dense_32 (Dense)            (None, 40)                2440      
                                                                 
 dense_33 (Dense)            (None, 20)                820       
                                                                 
 dense_34 (Dense)            (None, 1)                 21        
                                                                 
Total params: 5,861
Trainable params: 5,861
Non-trainable params: 0
_________________________________________________________________


In [40]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights_opt_3.{epoch:02d}.hdf5"

In [41]:
# Compile the model
nn_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5
)

# Train the model
fit_model = nn_3.fit(X_train_scaled, y_train, epochs=200, callbacks=[cp_callback])

Epoch 1/200
804/804 [==============================] - 2s 1ms/step - loss: 0.5692 - accuracy: 0.7222
Epoch 2/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5555 - accuracy: 0.7305
Epoch 3/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5528 - accuracy: 0.7298
Epoch 4/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5507 - accuracy: 0.7322
Epoch 5/200
791/804 [============================>.] - ETA: 0s - loss: 0.5505 - accuracy: 0.7313
Epoch 5: saving model to checkpoints\weights_opt_3.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5502 - accuracy: 0.7318
Epoch 6/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5492 - accuracy: 0.7325
Epoch 7/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5487 - accuracy: 0.7324
Epoch 8/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5473 - accuracy: 0.7339
Epoch 9/200
804/804 [===========================

804/804 [==============================] - 1s 1ms/step - loss: 0.5371 - accuracy: 0.7406
Epoch 64/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5369 - accuracy: 0.7397
Epoch 65/200
778/804 [============================>.] - ETA: 0s - loss: 0.5365 - accuracy: 0.7405
Epoch 65: saving model to checkpoints\weights_opt_3.65.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5368 - accuracy: 0.7402
Epoch 66/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5368 - accuracy: 0.7404
Epoch 67/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5369 - accuracy: 0.7409
Epoch 68/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5367 - accuracy: 0.7408
Epoch 69/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5359 - accuracy: 0.7415
Epoch 70/200
800/804 [============================>.] - ETA: 0s - loss: 0.5364 - accuracy: 0.7407
Epoch 70: saving model to checkpoints\weights_opt_3.70.h

804/804 [==============================] - 1s 1ms/step - loss: 0.5336 - accuracy: 0.7418
Epoch 126/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5338 - accuracy: 0.7417
Epoch 127/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5339 - accuracy: 0.7400
Epoch 128/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5338 - accuracy: 0.7411
Epoch 129/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5339 - accuracy: 0.7421
Epoch 130/200
794/804 [============================>.] - ETA: 0s - loss: 0.5338 - accuracy: 0.7415
Epoch 130: saving model to checkpoints\weights_opt_3.130.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5338 - accuracy: 0.7413
Epoch 131/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5332 - accuracy: 0.7415
Epoch 132/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5334 - accuracy: 0.7411
Epoch 133/200
804/804 [====================

804/804 [==============================] - 1s 1ms/step - loss: 0.5321 - accuracy: 0.7419
Epoch 188/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5324 - accuracy: 0.7418
Epoch 189/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5327 - accuracy: 0.7427
Epoch 190/200
782/804 [============================>.] - ETA: 0s - loss: 0.5329 - accuracy: 0.7419
Epoch 190: saving model to checkpoints\weights_opt_3.190.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5328 - accuracy: 0.7419
Epoch 191/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5325 - accuracy: 0.7416
Epoch 192/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5324 - accuracy: 0.7419
Epoch 193/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5326 - accuracy: 0.7415
Epoch 194/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5326 - accuracy: 0.7417
Epoch 195/200
792/804 [====================

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5661 - accuracy: 0.7245 - 428ms/epoch - 2ms/step
Loss: 0.5661494135856628, Accuracy: 0.7245481014251709


In [43]:
# Export our model to HDF5 file
nn_3.save("AlphabetSoupCharity_Optimization.h5")